<h1><center>Machine Learning Data Preprocessing - Regression  in seconds

# Competition Description
In this competition, Kaggle is challenging you to build a model that predicts the total ride duration of taxi trips in New York City. Your primary dataset is one released by the NYC Taxi and Limousine Commission, which includes pickup time, geo-coordinates, number of passengers, and several other variables.

# Data Resource
The dataset is downloaded from [Kaggle.com](https://www.kaggle.com/c/nyc-taxi-trip-duration/data). 

## Data fields
id - a unique identifier for each trip

vendor_id - a code indicating the provider associated with the trip record

pickup_datetime - date and time when the meter was engaged

dropoff_datetime - date and time when the meter was disengaged

passenger_count - the number of passengers in the vehicle (driver entered value)

pickup_longitude - the longitude where the meter was engaged

pickup_latitude - the latitude where the meter was engaged

dropoff_longitude - the longitude where the meter was disengaged

dropoff_latitude - the latitude where the meter was disengaged

store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip

trip_duration - duration of the trip in seconds

In [1]:
import warnings

# Ignore warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import gdown

In [5]:
import tensorflow as tf
from tensorflow import keras

In [7]:
# The random seed
random_seed = 42

# Set random seed in tensorflow
tf.random.set_seed(random_seed)

# Set random seed in numpy
import numpy as np
np.random.seed(random_seed)

# Data Preprocessing

## Part 1: Loading Data

In [8]:
# For the test dataset
url_test = 'https://drive.google.com/uc?id=1BKjRMYE8ZI0NDw8BU44YZqSCWqg3lIAv'
output_test = 'test_data.csv'
gdown.download(url_test, output_test, quiet=False)
df_raw_test = pd.read_csv(output_test)

# Set a deep copy for df_test_raw
df_test = df_raw_test.copy(deep=True)


# For the training dataset, convert the shareable link to a direct download link
url_train = 'https://drive.google.com/uc?id=1md8A0AUCzmvDv6dFrhF0G9E-6fTva72d'
output_train = 'train_data.csv'
gdown.download(url_train, output_train, quiet=False)
df_raw_train = pd.read_csv(output_train)

# Set a deep copy for df_train_raw
df_train = df_raw_train.copy(deep=True)

target = 'trip_duration'

Downloading...
From: https://drive.google.com/uc?id=1BKjRMYE8ZI0NDw8BU44YZqSCWqg3lIAv
To: C:\Users\yangy\iCloudDrive\Personal Life\Jobs in America\test_data.csv
100%|██████████| 70.8M/70.8M [00:03<00:00, 18.3MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1md8A0AUCzmvDv6dFrhF0G9E-6fTva72d
From (redirected): https://drive.google.com/uc?id=1md8A0AUCzmvDv6dFrhF0G9E-6fTva72d&confirm=t&uuid=0d9ecc56-5887-45ce-8d3f-702d4b9d6542
To: C:\Users\yangy\iCloudDrive\Personal Life\Jobs in America\train_data.csv
100%|██████████| 201M/201M [00:11<00:00, 17.4MB/s] 


In [9]:
# Print the dimension of the data_train
pd.DataFrame([[df_train.shape[0], df_train.shape[1]]], columns = ['#row', '#column'])

,#row,#column
0,1458644,11


In [10]:
# Print the dimension of the data_test
pd.DataFrame([[df_test.shape[0], df_test.shape[1]]], columns = ['#row', '#column'])

,#row,#column
0,625134,9


In [11]:
# Print the first 5 rows of df_train
print(df_train.head(5))

          id  vendor_id      pickup_datetime     dropoff_datetime  \
0  id2875421          2  2016-03-14 17:24:55  2016-03-14 17:32:30   
1  id2377394          1  2016-06-12 00:43:35  2016-06-12 00:54:38   
2  id3858529          2  2016-01-19 11:35:24  2016-01-19 12:10:48   
3  id3504673          2  2016-04-06 19:32:31  2016-04-06 19:39:40   
4  id2181028          2  2016-03-26 13:30:55  2016-03-26 13:38:10   

   passenger_count  pickup_longitude  pickup_latitude  dropoff_longitude  \
0                1        -73.982155        40.767937         -73.964630   
1                1        -73.980415        40.738564         -73.999481   
2                1        -73.979027        40.763939         -74.005333   
3                1        -74.010040        40.719971         -74.012268   
4                1        -73.973053        40.793209         -73.972923   

   dropoff_latitude store_and_fwd_flag  trip_duration  
0         40.765602                  N            455  
1         40.731

In [12]:
# Print the first 5 rows of df_test
print(df_test.head(5))

          id  vendor_id      pickup_datetime  passenger_count  \
0  id3004672          1  2016-06-30 23:59:58                1   
1  id3505355          1  2016-06-30 23:59:53                1   
2  id1217141          1  2016-06-30 23:59:47                1   
3  id2150126          2  2016-06-30 23:59:41                1   
4  id1598245          1  2016-06-30 23:59:33                1   

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0        -73.988129        40.732029         -73.990173         40.756680   
1        -73.964203        40.679993         -73.959808         40.655403   
2        -73.997437        40.737583         -73.986160         40.729523   
3        -73.956070        40.771900         -73.986427         40.730469   
4        -73.970215        40.761475         -73.961510         40.755890   

  store_and_fwd_flag  
0                  N  
1                  N  
2                  N  
3                  N  
4                  N  


## Part 2: Splitting the data

In [13]:
from sklearn.model_selection import train_test_split

# Split the df_train into training (80%) and validation (20%) data
df_train, df_val = train_test_split(df_train, train_size=0.8, random_state=random_seed)

# Reset the index
df_train, df_val = df_train.reset_index(drop=True), df_val.reset_index(drop=True)

In [14]:
# Print the dimension of df_train
pd.DataFrame([[df_train.shape[0], df_train.shape[1]]], columns = ['#row', '#column'])

,#row,#column
0,1166915,11


In [15]:
# Print the dimension of df_val
pd.DataFrame([[df_val.shape[0], df_val.shape[1]]], columns=['#row', '#column'])

,#row,#column
0,291729,11


## Part 3: Handling uncommon features

### 3.1 Identifying the common features

In [16]:
# Create the function 'common_var_checker'
def common_var_checker(df_train, df_val, df_test, target):
      df_common_var =  pd.DataFrame(np.intersect1d(np.intersect1d(df_train.columns, df_val.columns), np.union1d(df_test.columns, [target])), columns=['common_var'])

      return df_common_var

# Utilize the common_var_checker function
df_common_var = common_var_checker(df_train, df_val, df_test, target)

# Print df_common_var
df_common_var

,common_var
0,dropoff_latitude
1,dropoff_longitude
2,id
3,passenger_count
4,pickup_datetime
5,pickup_latitude
6,pickup_longitude
7,store_and_fwd_flag
8,trip_duration
9,vendor_id


### 3.2 Identifying the uncommon features

In [17]:
# Get the features in the traing data but not in test data
uncommon_feature_train_not_val_test = np.setdiff1d(df_train.columns, df_common_var['common_var'])

# Print the uncommon features
pd.DataFrame(uncommon_feature_train_not_val_test, columns=['uncommon_feature'])

,uncommon_feature
0,dropoff_datetime


In [18]:
# Get the features in the test data but not in training data
uncommon_feature_test_not_train_val=np.setdiff1d(df_test.columns, df_common_var['common_var'])

# Print the uncommon features
pd.DataFrame(uncommon_feature_test_not_train_val, columns=['uncommon feature'])

,uncommon feature


In [19]:
# Get the features in the val data but not in training and validation data
uncommon_feature_val_not_train_test = np.setdiff1d(df_val.columns, df_common_var['common_var'])

# Print the uncommon features
pd.DataFrame(uncommon_feature_val_not_train_test, columns=['uncommon feature'])

,uncommon feature
0,dropoff_datetime


### 3.3 Removing uncommon features

In [20]:
# Removing uncommon features from train, val, and test dataset
df_train = df_train.drop(columns=uncommon_feature_train_not_val_test)

# Print the first 5 rows of new df_train
df_train.head(5)

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id0458976,2,2016-06-29 18:21:02,1,-73.862762,40.768822,-73.891701,40.746689,N,1133
1,id0434613,2,2016-04-25 13:03:26,1,-73.958038,40.783237,-73.975510,40.760853,N,887
2,id3809234,2,2016-05-07 12:36:09,1,-73.969460,40.785519,-73.989243,40.771748,N,686
3,id1203705,1,2016-05-14 18:44:17,1,-73.981743,40.736549,-73.998352,40.726440,N,818
4,id1896645,2,2016-04-10 22:51:25,1,-73.977913,40.752609,-73.975647,40.733139,N,951


In [21]:
# Removing uncommon features from validation dataset
df_val = df_val.drop(columns=uncommon_feature_val_not_train_test)

# Print the first 5 rows of new df_val
df_val.head(5)

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2793718,2,2016-06-08 07:36:19,1,-73.985611,40.735943,-73.980331,40.760468,N,1040
1,id3485529,2,2016-04-03 12:58:11,1,-73.978394,40.764351,-73.991623,40.749859,N,827
2,id1816614,2,2016-06-05 02:49:13,5,-73.989059,40.744389,-73.973381,40.748692,N,614
3,id1050851,2,2016-05-05 17:18:27,2,-73.990326,40.731136,-73.991264,40.748917,N,867
4,id0140657,1,2016-05-12 17:43:38,4,-73.789497,40.646675,-73.987137,40.759232,N,4967


In [22]:
# Removing uncommon feature from df_test
df_test = df_test.drop(columns=uncommon_feature_test_not_train_val)

# Print the first 5 rows of df_test
df_test.head(5)

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


## Part 4: Handling identifiers

### 4.1 Building an id_checker function

In [23]:
def id_checker(df, dtype='float'):
      df_id = df[[var for var in df.columns
                  # Set condition to find the unique id
                  if (df[var].dtype != dtype
                  # the second condition is to find the number of unique value is equal to other columns without null/na value
                  and df[var].nunique(dropna=True) == df[var].notnull().sum())]]
      return df_id

In [24]:
# Set a total dataset df
df = pd.concat([df_train, df_val, df_test], sort=False)

# To search the df_id in the dataset df
df_id = id_checker(df)

# Print df_id
df_id

,id
0,id0458976
1,id0434613
2,id3809234
3,id1203705
4,id1896645
...,...
625129,id3008929
625130,id3700764
625131,id2568735
625132,id1384355


In [25]:
# Removing the df_id from df_train
df_train.drop(columns=np.intersect1d(df_id.columns, df_train.columns), inplace=True)

# Removing the df_id from df_val
df_val.drop(columns=np.intersect1d(df_id.columns, df_val.columns), inplace=True)

# Removing the df_id from df_test
df_test.drop(columns=np.intersect1d(df_id.columns, df_test.columns), inplace=True)

In [26]:
# Print the new df_train
df_train.head(5)

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,2,2016-06-29 18:21:02,1,-73.862762,40.768822,-73.891701,40.746689,N,1133
1,2,2016-04-25 13:03:26,1,-73.958038,40.783237,-73.975510,40.760853,N,887
2,2,2016-05-07 12:36:09,1,-73.969460,40.785519,-73.989243,40.771748,N,686
3,1,2016-05-14 18:44:17,1,-73.981743,40.736549,-73.998352,40.726440,N,818
4,2,2016-04-10 22:51:25,1,-73.977913,40.752609,-73.975647,40.733139,N,951


In [27]:
# Print the new df_val
df_val.head(5)

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,2,2016-06-08 07:36:19,1,-73.985611,40.735943,-73.980331,40.760468,N,1040
1,2,2016-04-03 12:58:11,1,-73.978394,40.764351,-73.991623,40.749859,N,827
2,2,2016-06-05 02:49:13,5,-73.989059,40.744389,-73.973381,40.748692,N,614
3,2,2016-05-05 17:18:27,2,-73.990326,40.731136,-73.991264,40.748917,N,867
4,1,2016-05-12 17:43:38,4,-73.789497,40.646675,-73.987137,40.759232,N,4967


In [28]:
# Print the new df_test
df_test.head(5)

,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


## Part 5: Handling data time variables

### 5.1 Building a new function to transfer datatime variables to new format

In [29]:
def datetime_transformer(df, datetime_vars):
      # Create a dictionary with key as datetime type and value as datetime type operator
      dict_ =  {'year' : lambda x : x.dt.year,
                     'month' : lambda x : x.dt.month,
                      'dayofweek' : lambda x : x.dt.dayofweek,
                     'day' : lambda x : x.dt.day,
                     'hour' : lambda x : x.dt.hour,
                     'minute' : lambda x : x.dt.minute,
                     'second' : lambda x : x.dt.second}
      # Make a copy of df
      df_datetime = df.copy(deep=True)

      # For every variable in datetime_vars
      for var in datetime_vars:
            df_datetime[var] = pd.to_datetime(df_datetime[var])

            # For every item (datetime_type and datetime_type_operator) in dict_
            for datetime_type, datetime_type_operator in dict_.items():
                  df_datetime[var + '_' + datetime_type] = datetime_type_operator(df_datetime[var])

      # Remove datetime_var from df_datetime
      df_datetime = df_datetime.drop(columns=datetime_vars)

      return df_datetime

### 5.2 Setting a variable datetime_vars

In [30]:
# for each dataset
datetime_vars = ['pickup_datetime']

# Call datetime_transformer on df_train
df_train = datetime_transformer(df_train, datetime_vars)

# Print the first rows of df_train
df_train.head(5)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_datetime_year,pickup_datetime_month,pickup_datetime_dayofweek,pickup_datetime_day,pickup_datetime_hour,pickup_datetime_minute,pickup_datetime_second
0,2,1,-73.862762,40.768822,-73.891701,40.746689,N,1133,2016,6,2,29,18,21,2
1,2,1,-73.958038,40.783237,-73.975510,40.760853,N,887,2016,4,0,25,13,3,26
2,2,1,-73.969460,40.785519,-73.989243,40.771748,N,686,2016,5,5,7,12,36,9
3,1,1,-73.981743,40.736549,-73.998352,40.726440,N,818,2016,5,5,14,18,44,17
4,2,1,-73.977913,40.752609,-73.975647,40.733139,N,951,2016,4,6,10,22,51,25


In [31]:
# Call datetime_transformer on df_val
df_val = datetime_transformer(df_val, datetime_vars)

# Print the first 5 rows of df_val
df_val.head(5)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_datetime_year,pickup_datetime_month,pickup_datetime_dayofweek,pickup_datetime_day,pickup_datetime_hour,pickup_datetime_minute,pickup_datetime_second
0,2,1,-73.985611,40.735943,-73.980331,40.760468,N,1040,2016,6,2,8,7,36,19
1,2,1,-73.978394,40.764351,-73.991623,40.749859,N,827,2016,4,6,3,12,58,11
2,2,5,-73.989059,40.744389,-73.973381,40.748692,N,614,2016,6,6,5,2,49,13
3,2,2,-73.990326,40.731136,-73.991264,40.748917,N,867,2016,5,3,5,17,18,27
4,1,4,-73.789497,40.646675,-73.987137,40.759232,N,4967,2016,5,3,12,17,43,38


In [32]:
# Call datetime_transformer on df_test
df_test = datetime_transformer(df_test, datetime_vars)

# Print the first 5 rows of df_test
df_test.head(5)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_datetime_year,pickup_datetime_month,pickup_datetime_dayofweek,pickup_datetime_day,pickup_datetime_hour,pickup_datetime_minute,pickup_datetime_second
0,1,1,-73.988129,40.732029,-73.990173,40.756680,N,2016,6,3,30,23,59,58
1,1,1,-73.964203,40.679993,-73.959808,40.655403,N,2016,6,3,30,23,59,53
2,1,1,-73.997437,40.737583,-73.986160,40.729523,N,2016,6,3,30,23,59,47
3,2,1,-73.956070,40.771900,-73.986427,40.730469,N,2016,6,3,30,23,59,41
4,1,1,-73.970215,40.761475,-73.961510,40.755890,N,2016,6,3,30,23,59,33


## Part 6: Handling missing data
### 6.1 Combining the training, validation, and testing data

In [33]:
df = pd.concat([df_train, df_val, df_test], sort=False)

### 6.2 Defining a function nan_checker

In [34]:
def nan_checker(df):

      # Get the dataframe of variables with NaN, their proportion, and the data type
      df_nan = pd.DataFrame([[var, df[var].isna().sum() / df.shape[0], df[var].dtype]
                             for var in df.columns if df[var].isna().sum()>0],
                            columns=['var', 'proportion', 'dtype'])

      # Sort df_nan in descending order of the proportion of NaN
      df_nan = df_nan.sort_values(by='proportion', ascending=False).reset_index(drop=True)

      return df_nan

### 6.3 Calling the function on each data

In [35]:
# Call the function nan_checker on df
df_nan = nan_checker(df)

# Print the first 5 rows of df
df_nan.head(5)

,var,proportion,dtype
0,trip_duration,0.3,float64


In [36]:
# Call the function nan_checker on df_train, df_val, and df_test
df_nan_train = nan_checker(df_train)

# Print the first 5 rows of df_train
df_nan_train.head(5)

,var,proportion,dtype


In [37]:
# Call the function nan_checker on df_val
df_nan_val = nan_checker(df_val)

# Print the first 5 rows of df_val
df_nan_val.head(5)

,var,proportion,dtype


In [38]:
# Call the function nan_checker on df_test
df_nan_test =  nan_checker(df_test)

# Print the first 5 rows of df_test
df_nan_test.head(5)

,var,proportion,dtype


In [39]:
# Print the unique data type of variables with NaN
pd.DataFrame(df_nan['dtype'].unique(), columns=['dtype'])

,dtype
0,float64


### 6.4 Checing the missing value

In [40]:
# Get the variables with missing values
df_miss = df_nan[df_nan['dtype'] == 'float64'].reset_index(drop=True)

# Print the df_miss
df_miss

,var,proportion,dtype
0,trip_duration,0.3,float64


### Summary
There is  0.3 NaN value in the df. When I check the NaN value in df_train, df_val, it shows no NaN value in both dataset. Therefore, it means only df_test has no the target value. It is common in the real world.  

## Part 7: Feature Engineering and Dimensionality Reductiontter.

### 7.1 Overview

In the NYC-taxi-trip-duration dataset, there are some features like pickup_latitude, pickup_longtitude, dropoff_latitude, dropoff_longtitude that can be reduced to distance. Also, I can transfer the pickup_datetime to two categorical data 'rush_nonrush' and 'weekday_weekend'. If the pickup time is on the rush time in one day, the trip duration maybe longer, and if the pickup day is in weekday, the trip duration maybe also longer.

Through the feature engineering, we can reduce the dimensionality and also make the prediction result better.

### 7.2 Transforming time series data feature

In [41]:
# Set a new column 'rush_nonrush' for df_train
df_train['rush_nonrush'] = df_train['pickup_datetime_hour'].apply(lambda x : 'rush' if 7<=x<19 else 'nonrush')

# Set a new column 'weekday_weekend' for df_train
df_train['weekday_weekend'] = df_train['pickup_datetime_dayofweek'].apply(lambda x: 'weekday' if x< 5 else 'weekend')

# Set a list drop_column
drop_column_train = df_train.filter(like = 'pickup_datetime_').columns

# Drop all pickup_datetime columns
df_train = df_train.drop(columns=drop_column_train )

# Print df_train
df_train.head(5)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,rush_nonrush,weekday_weekend
0,2,1,-73.862762,40.768822,-73.891701,40.746689,N,1133,rush,weekday
1,2,1,-73.958038,40.783237,-73.975510,40.760853,N,887,rush,weekday
2,2,1,-73.969460,40.785519,-73.989243,40.771748,N,686,rush,weekend
3,1,1,-73.981743,40.736549,-73.998352,40.726440,N,818,rush,weekend
4,2,1,-73.977913,40.752609,-73.975647,40.733139,N,951,nonrush,weekend


In [42]:
# Set a new column 'rush_nonrush' for df_val
df_val['rush_nonrush'] = df_val['pickup_datetime_hour'].apply(lambda x : 'rush' if 7<=x<19 else 'nonrush')

# Set a new column 'weekday_weekend' for df
df_val['weekday_weekend'] = df_val['pickup_datetime_dayofweek'].apply(lambda x: 'weekday' if x< 5 else 'weekend')

# Set a list drop_column
drop_column_val = df_val.filter(like = 'pickup_datetime_').columns

# Drop all pickup_datetime columns
df_val = df_val.drop(columns=drop_column_train )

# Print df_train
df_val.head(5)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,rush_nonrush,weekday_weekend
0,2,1,-73.985611,40.735943,-73.980331,40.760468,N,1040,rush,weekday
1,2,1,-73.978394,40.764351,-73.991623,40.749859,N,827,rush,weekend
2,2,5,-73.989059,40.744389,-73.973381,40.748692,N,614,nonrush,weekend
3,2,2,-73.990326,40.731136,-73.991264,40.748917,N,867,rush,weekday
4,1,4,-73.789497,40.646675,-73.987137,40.759232,N,4967,rush,weekday


In [43]:
# Set a new column 'rush_nonrush' for df_test
df_test['rush_nonrush'] = df_test['pickup_datetime_hour'].apply(lambda x : 'rush' if 7<=x<19 else 'nonrush')

# Set a new column 'weekday_weekend' for df_test
df_test['weekday_weekend'] = df_test['pickup_datetime_dayofweek'].apply(lambda x: 'weekday' if x< 5 else 'weekend')

# Set a list drop_column
drop_column_test = df_test.filter(like = 'pickup_datetime_').columns

# Drop all pickup_datetime columns
df_test = df_test.drop(columns=drop_column_train )

# Print df_train
df_test.head(5)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,rush_nonrush,weekday_weekend
0,1,1,-73.988129,40.732029,-73.990173,40.756680,N,nonrush,weekday
1,1,1,-73.964203,40.679993,-73.959808,40.655403,N,nonrush,weekday
2,1,1,-73.997437,40.737583,-73.986160,40.729523,N,nonrush,weekday
3,2,1,-73.956070,40.771900,-73.986427,40.730469,N,nonrush,weekday
4,1,1,-73.970215,40.761475,-73.961510,40.755890,N,nonrush,weekday


### 7.3 Distance Feature Engineering

Create one column called 'distance' from the four columns 'pickup_longtitude', ..., 'dropoff_latitude'.

In [44]:
# Create a new function to calculate the distance
from math import radians, sin, cos, sqrt, atan2
def haversine_distance(row):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = radians(row['pickup_latitude']), radians(row['pickup_longitude'])
    lat2, lon2 = radians(row['dropoff_latitude']), radians(row['dropoff_longitude'])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    radius_of_earth = 6371  # Earth's radius in kilometers
    distance = radius_of_earth * c

    return distance

In [45]:
# Call the function on df_train
df_train['distance'] = df_train.apply(haversine_distance, axis=1)

# Drop the four columns
df_train = df_train.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

# Display the first 5 rows of df_train
df_train.head(5)

,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,rush_nonrush,weekday_weekend,distance
0,2,1,N,1133,rush,weekday,3.463778
1,2,1,N,887,rush,weekday,2.891365
2,2,1,N,686,rush,weekend,2.262636
3,1,1,N,818,rush,weekend,1.795030
4,2,1,N,951,nonrush,weekend,2.173389


In [46]:
# Call the function on df_val
df_val['distance'] = df_val.apply(haversine_distance, axis=1)

# Drop the four columns
df_val = df_val.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

# Display the first 5 rows of df_val
df_val.head(5)

,vendor_id,passenger_count,store_and_fwd_flag,trip_duration,rush_nonrush,weekday_weekend,distance
0,2,1,N,1040,rush,weekday,2.763050
1,2,1,N,827,rush,weekend,1.959178
2,2,5,N,614,nonrush,weekend,1.404772
3,2,2,N,867,rush,weekday,1.978660
4,1,4,N,4967,rush,weekday,20.837745


In [47]:
# Call the function on df_test
df_test['distance'] = df_test.apply(haversine_distance, axis=1)

# Drop the four columns
df_test = df_test.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

# Display the first 5 rows of df_test
df_test.head(5)

,vendor_id,passenger_count,store_and_fwd_flag,rush_nonrush,weekday_weekend,distance
0,1,1,N,nonrush,weekday,2.746426
1,1,1,N,nonrush,weekday,2.759239
2,1,1,N,nonrush,weekday,1.306155
3,2,1,N,nonrush,weekday,5.269088
4,1,1,N,nonrush,weekday,0.960842


## Part 8: Encoding the categorical data

### 8.1 Defining a cat_var_checker

In [48]:
def cat_var_checker(df, dtype='object'):
      # Get the dataframe of categorical variables and their number of unique value
      df_cat = pd.DataFrame([[var, df[var].nunique(dropna=False)]
                                              for var in df.columns if df[var].dtype == dtype],
                                              columns=['var', 'nunique'])
      # Sort df_cat in descending order by the number of unique values
      df_cat =  df_cat.sort_values(by='nunique', ascending=False).reset_index(drop=True)

      return df_cat

### 8.2 Calling cat_var_checker function

In [49]:
# Check for the categorical data in df_train
df_cat_train = cat_var_checker(df_train)

# Print the first 5 rows of df_cat_train
df_cat_train.head(5)

,var,nunique
0,store_and_fwd_flag,2
1,rush_nonrush,2
2,weekday_weekend,2


In [50]:
# Check for the categorical data in df_val
df_cat_val = cat_var_checker(df_val)

# Print the first 5 rows of df_cat_val
df_cat_val.head(5)

,var,nunique
0,store_and_fwd_flag,2
1,rush_nonrush,2
2,weekday_weekend,2


In [51]:
# Check for the categorical data in df_test
df_cat_test = cat_var_checker(df_test)

# Print the first 5 rows of df_cat_val
df_cat_test.head(5)

,var,nunique
0,store_and_fwd_flag,2
1,rush_nonrush,2
2,weekday_weekend,2


### 8.3 Encoding the data

In [52]:
# One-hot-encode the categorical features in the df-val
df_train = pd.get_dummies(df_train, columns=np.setdiff1d(df_cat_train['var'], [target]))

# Print the first 5 rows of df_train
df_train.head(5)

,vendor_id,passenger_count,trip_duration,distance,rush_nonrush_nonrush,rush_nonrush_rush,store_and_fwd_flag_N,store_and_fwd_flag_Y,weekday_weekend_weekday,weekday_weekend_weekend
0,2,1,1133,3.463778,False,True,True,False,True,False
1,2,1,887,2.891365,False,True,True,False,True,False
2,2,1,686,2.262636,False,True,True,False,False,True
3,1,1,818,1.795030,False,True,True,False,False,True
4,2,1,951,2.173389,True,False,True,False,False,True


In [53]:
# One-hot-encode the categorical features in the df-val
df_val = pd.get_dummies(df_val, columns=np.setdiff1d(df_cat_val['var'], [target]))

# Print the first 5 rows of df_val
df_val.head(5)

,vendor_id,passenger_count,trip_duration,distance,rush_nonrush_nonrush,rush_nonrush_rush,store_and_fwd_flag_N,store_and_fwd_flag_Y,weekday_weekend_weekday,weekday_weekend_weekend
0,2,1,1040,2.763050,False,True,True,False,True,False
1,2,1,827,1.959178,False,True,True,False,False,True
2,2,5,614,1.404772,True,False,True,False,False,True
3,2,2,867,1.978660,False,True,True,False,True,False
4,1,4,4967,20.837745,False,True,True,False,True,False


In [54]:
# One-hot-encode the categorical features in the df-val
df_test = pd.get_dummies(df_test, columns=np.setdiff1d(df_cat_test['var'], [target]))

# Print the first 5 rows of df_val
df_test.head(5)

,vendor_id,passenger_count,distance,rush_nonrush_nonrush,rush_nonrush_rush,store_and_fwd_flag_N,store_and_fwd_flag_Y,weekday_weekend_weekday,weekday_weekend_weekend
0,1,1,2.746426,True,False,True,False,True,False
1,1,1,2.759239,True,False,True,False,True,False
2,1,1,1.306155,True,False,True,False,True,False
3,2,1,5.269088,True,False,True,False,True,False
4,1,1,0.960842,True,False,True,False,True,False


## Part 9: Splitting the feature and target

In [55]:
# Get the feature matrix
X_train = df_train[np.setdiff1d(df_train.columns, [target])].values
X_val = df_val[np.setdiff1d(df_val.columns, [target])].values
X_test = df_test[np.setdiff1d(df_test.columns, [target])].values

# Get the target matrix
Y_train = df_train[target].values
Y_val = df_val[target].values

## Part 10: Scaling the data

In [56]:
from sklearn.preprocessing import StandardScaler

# The StandardScaler
ss = StandardScaler()

### 10.1 Standardizing the features

In [57]:
# Standardize the training data
X_train = ss.fit_transform(X_train)

# Standardize the validation data
X_val = ss.transform(X_val)

# Standardize the test data
X_test = ss.transform(X_test)

### 10.2 Standardizing the target

In [58]:
# Standardize the training data target
Y_train = ss.fit_transform(Y_train.reshape(-1, 1)).reshape(-1)

# Standardize the validation data target
Y_val = ss.transform(Y_val.reshape(-1, 1)).reshape(-1)

## Display the final dataset

In [64]:
# Display the final X_train
X_train[0:5]

array([[ 0.00537456, -0.50537005, -0.85241374,  0.85241374,  0.07469793,
        -0.07469793,  0.93209347,  0.63156822, -0.63156822],
       [-0.12650497, -0.50537005, -0.85241374,  0.85241374,  0.07469793,
        -0.07469793,  0.93209347,  0.63156822, -0.63156822],
       [-0.27135898, -0.50537005, -0.85241374,  0.85241374,  0.07469793,
        -0.07469793,  0.93209347, -1.58336022,  1.58336022],
       [-0.37909184, -0.50537005, -0.85241374,  0.85241374,  0.07469793,
        -0.07469793, -1.07285377, -1.58336022,  1.58336022],
       [-0.29192085, -0.50537005,  1.17313924, -1.17313924,  0.07469793,
        -0.07469793,  0.93209347, -1.58336022,  1.58336022]])

In [65]:
# Display the final X_val
X_val[0:5]

array([[-0.15606773, -0.50537005, -0.85241374,  0.85241374,  0.07469793,
        -0.07469793,  0.93209347,  0.63156822, -0.63156822],
       [-0.34127325, -0.50537005, -0.85241374,  0.85241374,  0.07469793,
        -0.07469793,  0.93209347, -1.58336022,  1.58336022],
       [-0.46900412,  2.53968633,  1.17313924, -1.17313924,  0.07469793,
        -0.07469793,  0.93209347, -1.58336022,  1.58336022],
       [-0.3367849 ,  0.25589405, -0.85241374,  0.85241374,  0.07469793,
        -0.07469793,  0.93209347,  0.63156822, -0.63156822],
       [ 4.0081981 ,  1.77842223, -0.85241374,  0.85241374,  0.07469793,
        -0.07469793, -1.07285377,  0.63156822, -0.63156822]])

In [66]:
# Display the final X_test
X_test[0:5]

array([[-0.15989771, -0.50537005,  1.17313924, -1.17313924,  0.07469793,
        -0.07469793, -1.07285377,  0.63156822, -0.63156822],
       [-0.15694567, -0.50537005,  1.17313924, -1.17313924,  0.07469793,
        -0.07469793, -1.07285377,  0.63156822, -0.63156822],
       [-0.49172455, -0.50537005,  1.17313924, -1.17313924,  0.07469793,
        -0.07469793, -1.07285377,  0.63156822, -0.63156822],
       [ 0.4213035 , -0.50537005,  1.17313924, -1.17313924,  0.07469793,
        -0.07469793,  0.93209347,  0.63156822, -0.63156822],
       [-0.57128204, -0.50537005,  1.17313924, -1.17313924,  0.07469793,
        -0.07469793, -1.07285377,  0.63156822, -0.63156822]])